### FOBSS Preprocessing Script
---
Notebook for trying first data preprocessing steps

### Selecting relevant part of Ri Jumps 15A Profile

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

cur_master = np.loadtxt('../data/fobss_data/data/Ri Jumps 25A/battery/Battery_Current.csv', delimiter=';')
cur_master = cur_master[:100000]

cur_inv = np.loadtxt('../data/fobss_data/data/Ri Jumps 25A/inverter/Inverter_Current.csv', delimiter=';')
cur_inv = cur_inv[:350000]

volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/Ri Jumps 25A/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
volt_slave_0_cell_4 = volt_slave_0_cell_4[:100000]

In [ ]:
plt.plot(cur_master[:,1]) 
cur_master.shape

In [ ]:
plt.plot(volt_slave_0_cell_4[:,4])
volt_slave_0_cell_4.shape

### Data Subsample Function

In [ ]:
def subsample(sequence, d_t):

    subsampled_sequence = np.array([])

    for i in range(0, len(sequence), d_t - 1) :
        
        if len(sequence) < i + d_t:
            # check if at end of sequence
            window = sequence[i:len(sequence)]
        else:
            window = sequence[i:i + d_t]
        
        # search for most frequent element in observed window
        uniqw, inverse = np.unique(window, return_inverse=True)
        frequent_element_index = np.bincount(inverse).argmax()
        frequent_element = window[frequent_element_index]
    
        # add most frequent element to subsample
        subsampled_sequence = np.append(subsampled_sequence, frequent_element)
        
    return subsampled_sequence

In [ ]:
def smoothing(sequence, d_t):

    subsampled_sequence = np.array([])

    for i in range(0, len(sequence), d_t - 1) :
        
        if len(sequence) < i + d_t:
            # check if at end of sequence
            window = sequence[i:len(sequence)]
            seq = np.ones((len(sequence) - i, 1))
        else:
            window = sequence[i:i + d_t]
            seq = np.ones((d_t, 1))
        
        # search for most frequent element in observed window
        uniqw, inverse = np.unique(window, return_inverse=True)
        frequent_element_index = np.bincount(inverse).argmax()
        frequent_element = window[frequent_element_index]
    
        # add most frequent element to subsample
        subsampled_sequence = np.append(subsampled_sequence, seq * frequent_element)
        
    return subsampled_sequence

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

cur_master = np.loadtxt('../data/fobss_data/data/Profile 10A/battery/Battery_Current.csv', delimiter=';')
cur_master = cur_master[500:1000]

volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/Profile 10A/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
volt_slave_0_cell_4 = volt_slave_0_cell_4[500:1000]

In [ ]:
plt.plot(volt_slave_0_cell_4[:,4])
volt_slave_0_cell_4.shape

In [ ]:
subsample = subsample(volt_slave_0_cell_4[:,4], 10)
plt.plot(subsample)
subsample.shape

In [ ]:
smoothing = smoothing(volt_slave_0_cell_4[:,4], 10)
plt.plot(smoothing)
smoothing.shape

### Subsampling and Smoothing using libraries

In [ ]:
from scipy import ndimage 

def subsample(sequence, d_sample):
    return sequence[::d_sample]

def smooth(sequence, sigma):
    return ndimage.filters.gaussian_filter(sequence, sigma)

# ------------------------------------------- Prepare Test Data -------------------------------------------
test_cur_master = np.loadtxt('../data/fobss_data/data/Ri Jumps 25A/battery/Battery_Current.csv', delimiter=';')
test_cur_master = test_cur_master[80000:100000,1]
test_volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/Ri Jumps 25A/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
test_volt_slave_0_cell_4 = test_volt_slave_0_cell_4[80000:100000, 4]

# subsample and smooth data 
# test_cur_master = subsample(test_cur_master, 10)
# test_cur_master = smooth(test_cur_master, 10)

test_volt_slave_0_cell_4 = subsample(test_volt_slave_0_cell_4, 10)
test_volt_slave_0_cell_4 = smooth(test_volt_slave_0_cell_4, 10)

plt.plot(test_volt_slave_0_cell_4[600:700])

### Align two arrays of different length

In [ ]:
from scipy import ndimage 

train_cur_inv = np.loadtxt('../data/fobss_data/data/Profile 10A Run 040618_2/inverter/Inverter_Current.csv', delimiter=';')
train_cur_inv = train_cur_inv[:,1]
train_volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/Profile 10A Run 040618_2/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
train_volt_slave_0_cell_4 = train_volt_slave_0_cell_4[:,4]
train_volt_slave_0_cell_4 = ndimage.filters.gaussian_filter(train_volt_slave_0_cell_4, 10)

factor = train_cur_inv.shape[0] / train_volt_slave_0_cell_4.shape[0]

train_cur_inv_ = list()
for i in range(len(train_volt_slave_0_cell_4)):
    train_cur_inv_.append(train_cur_inv[int(np.round(i * factor))])

train_cur_inv_resized = np.array(train_cur_inv_)

# plot graph
plt.subplots(figsize = (15,15))
plt.subplot(2,1,1)
plt.plot(train_cur_inv_resized, label="Inverter Current")
plt.legend()
plt.subplot(2,1,2)
plt.plot(train_volt_slave_0_cell_4, label="Run 1", color='red')
plt.legend()